In [2]:
import pandas as pd

data = {
    "Temperature_C": [35, 22, 18, 10, 25, 30, 15, 5, 28, 20],
    "Weather": ["Sunny", "Cloudy", "Rainy", "Cold", "Clear", "Hot", "Cold", "Freezing", "Warm", "Windy"],
    "Feedback": [
        "Too hot to do anything",
        "Nice breeze, feels good",
        "Wet and dull day",
        "Freezing, not feeling good",
        "Pleasant and fresh",
        "Sweating and tired",
        "Shivering all day",
        "Extremely cold morning",
        "Warm and energetic",
        "A bit windy but okay"
    ],
    "Sentiment_Label": [-1, 1, 0, -1, 1, -1, -1, -1, 1, 0],
    "Recommended_Activity": [
        "Stay indoors",
        "Go for a walk",
        "Read a book",
        "Stay indoors",
        "Play outside",
        "Stay hydrated indoors",
        "Wear warm clothes indoors",
        "Stay indoors with heater",
        "Go cycling",
        "Go for a short walk"
    ]
}


df = pd.DataFrame(data)


df.to_csv("temperature_activity_dataset.csv", index=False)

print("CSV file created successfully.")


CSV file created successfully.


In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv("temperature_activity_dataset.csv")

def clean_text(text):
    return ''.join(c.lower() for c in text if c.isalnum() or c == ' ')

df['Clean_Feedback'] = df['Feedback'].apply(clean_text)


vocab = set()
for text in df['Clean_Feedback']:
    vocab.update(text.split())

vocab = sorted(vocab)
word_index = {word: i for i, word in enumerate(vocab)}

def text_to_bow(text):
    vec = [0] * len(vocab)
    for word in text.split():
        if word in word_index:
            vec[word_index[word]] += 1
    return vec

X_text = [text_to_bow(text) for text in df['Clean_Feedback']]
y_sent = df['Sentiment_Label']

sentiment_model = LogisticRegression()
sentiment_model.fit(X_text, y_sent)


le_weather = LabelEncoder()
df['Weather_Code'] = le_weather.fit_transform(df['Weather'])

X_act = df[['Temperature_C', 'Weather_Code']]
y_act = df['Recommended_Activity']

activity_model = LogisticRegression(max_iter=1000)
activity_model.fit(X_act, y_act)


LogisticRegression(max_iter=1000)

In [5]:
def main():
    while True:
        print("\n🌡️ Temperature Feedback Analyzer")
        print("1. View data")
        print("2. Predict sentiment from feedback")
        print("3. Recommend activity based on temperature & weather")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == '1':
            display(df[['Temperature_C', 'Weather', 'Feedback', 'Sentiment_Label', 'Recommended_Activity']])

        elif choice == '2':
            feedback = input("Enter feedback: ")
            cleaned = clean_text(feedback)
            bow = text_to_bow(cleaned)
            pred = sentiment_model.predict([bow])[0]
            label = "Positive" if pred == 1 else "Neutral" if pred == 0 else "Negative"
            print(f"Predicted Sentiment: {label}")

        elif choice == '3':
            try:
                temp = float(input("Enter temperature (°C): "))
                weather = input(f"Enter weather type {list(le_weather.classes_)}: ")
                weather_code = le_weather.transform([weather])[0]
                pred = activity_model.predict([[temp, weather_code]])[0]
                print(f"Recommended Activity: {pred}")
            except Exception as e:
                print("Error in input. Please try again.")

        elif choice == '4':
            print("Exiting. Stay safe!")
            break

        else:
            print("Invalid choice. Choose between 1-4.")

if __name__ == "__main__":
    main()


🌡️ Temperature Feedback Analyzer
1. View data
2. Predict sentiment from feedback
3. Recommend activity based on temperature & weather
4. Exit
Exiting. Stay safe!
